### Webscrapping with Pandas/BeautifulSoup + ODBC on SQL Server with pyodbc

#### Source : Wikipedia / Rock Climbing and Mountaineering

This page is discribed as a list of climbers and mountaineers who are notable for their activities in mountaineering (including alpine climbing), rock climbing (including aid climbing, free climbing, bouldering, speed climbing and competition climbing) or in ice climbing (including mixed climbing).

Extracted tables are NOT list of climbers and mountaineers BUT data about Rock Climbing and Mountaineering 

In [24]:
import numpy as np
import pandas as pd 
# Beautifulsoup already installed into Anaconda environment

# Librairy PyODBC for SQL Server connection
import os
import pyodbc
# setup SQL Server connection session
server = '-PCSJN\DATAVIZ'  
database = 'Climbing_Staging'
connection_string= (
                    'DRIVER={SQL Server};SERVER='
                     + server
                     + ';DATABASE='
                     + database 
                     + ';Trusted_Connection=yes'                  
                    )

# Function for injecting dataframe content into SQL Server Table
def template_SQL_insert_values(table_name,dict):
    
    columns = "("
    for k in dict.keys():
        columns += str(k) + ','
    columns = columns[:-1] ### remove last comma
    columns += ")"
    
    req = f"""INSERT INTO {table_name} {columns} VALUES {tuple(dict.values())}"""
    return req

In [25]:
###un premier url
Wiki_Notable_Climbers_and_Mountaineers = "https://en.wikipedia.org/wiki/List_of_climbers_and_mountaineers"
### récuperer toutes les tables html vu sur le site (url)
Dataset_Wiki_Notable_Climbers_and_Mountaineers = pd.read_html(Wiki_Notable_Climbers_and_Mountaineers)
type(Dataset_Wiki_Notable_Climbers_and_Mountaineers), len(Dataset_Wiki_Notable_Climbers_and_Mountaineers)

(list, 10)

### on récupère 1 list de 10 élts

In [26]:
type(Dataset_Wiki_Notable_Climbers_and_Mountaineers[0])

pandas.core.frame.DataFrame

### Chaque élt est un DataFrame Pandas
### Quel est le contenu chacun des 10 dataframes ? Lesquelles nous interessent ?

In [27]:
Dataset_Wiki_Notable_Climbers_and_Mountaineers[0]

,0
0,Part of a series on
1,Climbing
2,"Reinhold Messner, the first mountaineer to cli..."
3,Lists
4,Climbers Piolet d'Or winners IFSC victories Eq...
5,Types of rock climbing
6,Aid Big wall Multi-pitch Bouldering Highball C...
7,Types of mountaineering
8,Alpine Mixed Via ferrata Himalayan Alpine styl...
9,Other types


[0] is about Types of RockClimbing and Mountaineering

In [28]:
Dataset_Wiki_Notable_Climbers_and_Mountaineers[1]

,vteClimbing,vteClimbing.1
0,Types,Rock climbingDisciplines Aid Big wall Multi-pi...
1,Rock climbing,Disciplines Aid Big wall Multi-pitch Boulderin...
2,Disciplines,Aid Big wall Multi-pitch Bouldering Highball C...
3,Rock types,Crack Face Roof Slab
4,Mountaineering,Alpine Via ferrata Himalayan Alpine style Expe...
5,Other,Buildering Canyoning Crane Grass Commercial Pa...
6,Lists,Alpine clubs Climbers Piolet d'Or winners IFSC...
7,Terminology,EquipmentRock Aider Daisy chain Anchor Ascende...
8,Equipment,Rock Aider Daisy chain Anchor Ascender Croll B...
9,Rock,Aider Daisy chain Anchor Ascender Croll Belay ...


[1] is about RockClimbing, Mountaineering Types, Equipments, Magazines, Videos, Companies, Organizations

In [29]:
Dataset_Wiki_Notable_Climbers_and_Mountaineers[2]

,0,1
0,Rock climbing,Disciplines Aid Big wall Multi-pitch Boulderin...
1,Disciplines,Aid Big wall Multi-pitch Bouldering Highball C...
2,Rock types,Crack Face Roof Slab
3,Mountaineering,Alpine Via ferrata Himalayan Alpine style Expe...
4,Other,Buildering Canyoning Crane Grass Commercial Pa...


In [47]:
Dataset_Wiki_Notable_Climbers_and_Mountaineers[2]

(5, 2)

[2] is about RockClimbing, Mountaineering and Rock Types

In [31]:
Dataset_Wiki_Notable_Climbers_and_Mountaineers[3]

,0,1
0,Disciplines,Aid Big wall Multi-pitch Bouldering Highball C...
1,Rock types,Crack Face Roof Slab


[3] is duplicate fo [2] > To Discard

In [32]:
Dataset_Wiki_Notable_Climbers_and_Mountaineers[4]

,0,1
0,Equipment,Rock Aider Daisy chain Anchor Ascender Croll B...
1,Rock,Aider Daisy chain Anchor Ascender Croll Belay ...
2,Ice and snow,Alpenstock Abalakov thread Bottled oxygen Cram...
3,Action,Abseiling Australian Dülfersitz Belaying Front...
4,Other,Ape index Approach shoe Clean Climbing route B...


[4] is about Equipments

In [34]:
Dataset_Wiki_Notable_Climbers_and_Mountaineers[5]

,0,1
0,Rock,Aider Daisy chain Anchor Ascender Croll Belay ...
1,Ice and snow,Alpenstock Abalakov thread Bottled oxygen Cram...


[5] is duplicate fo [4] > To Discard

In [35]:
Dataset_Wiki_Notable_Climbers_and_Mountaineers[6]

,0,1
0,Magazines and journals,Alpine Journal Alpinist American Alpine Journa...
1,Non-fiction films,The Alpinist El Capitan The Dawn Wall Free Sol...


[6] is about Media (Journals and Videos)

In [39]:
Dataset_Wiki_Notable_Climbers_and_Mountaineers[7]

,0,1
0,National,Access Fund Alpine Club Alpine Club of Canada ...
1,International,International Federation of Sport Climbing (IF...


[7] is about Organizations

In [41]:
Dataset_Wiki_Notable_Climbers_and_Mountaineers[8]

,vteExploration,vteExploration.1
0,Exploration by environment,Canopy Cave Cave diving Desert Ocean Deep-sea
1,Exploration by region,Chronology of European exploration of Asia His...
2,Timelines,Chinese exploration Timeline of European explo...
3,Expeditions,1928 Great Barrier Reef expedition Austrian ex...
4,Lists of explorers,By country of origin French Italian Portuguese...
5,By country of origin,French Italian Portuguese Spanish Romanian Rus...
6,By environment,Circumnavigators Climbers Polar explorers Unde...


[8] is about Explorations

In [42]:
Dataset_Wiki_Notable_Climbers_and_Mountaineers[9]

,0,1
0,By country of origin,French Italian Portuguese Spanish Romanian Rus...
1,By environment,Circumnavigators Climbers Polar explorers Unde...


[9] is duplicate fo [8] > To Discard